In [1]:
from qiskit import QuantumCircuit, transpile, execute, IBMQ
from states import Key, Ciphertext
from global_parameters import GlobalParameters
from encryption_circuit import encrypt
from verification_circuit import verify_deletion_counts
from decryption_circuit import create_decryption_circuit, decrypt_results
from deletion_circuit import delete
from attack_circuit import breidbart_measurement
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeMontreal
from qiskit.circuit import qpy_serialization
from utils import export_counts, random_bit_string
import os
import shutil

In [2]:
# Qiskit parameters
provider = IBMQ.load_account()

# Simulator
noise_model = FakeMontreal()
backend = AerSimulator.from_backend(noise_model)

# Real device - uncomment, might be a different way to access
# backend = provider.get_backend("ibmq_belem")

optimization_level = 0
shots = 1000

# Create data directory
if not os.path.exists("data"):
    os.mkdir("data")

In [3]:
# Utility function to run a circuit, and return the result counts
def run_and_measure(circuit: QuantumCircuit, draw: bool = False) -> dict[str, int]:
    """Transpiles and runs the QuantumCircuit, and returns the resulting counts of the execution."""
    if draw:
        print(circuit.draw())
    transpiled_circuit = transpile(
        circuit, backend=backend, optimization_level=optimization_level)
    if draw:
        print(transpiled_circuit.draw())
    result = backend.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    # Reverse the string since the most significant qubit is at the 0th index of the measurement string
    reversed_keys_dict = {}
    for key, value in counts.items():
        reversed_keys_dict[key[::-1]] = value
    return reversed_keys_dict

In [4]:
# Encrypt 
global_params = GlobalParameters.generate_from_lambda(1)
key = Key.generate_key(global_params)
message = random_bit_string(global_params.n)
ciphertext = encrypt(message, key, global_params)
original_circuit = ciphertext.circuit.copy() # We might modify this circuit in subsequent steps

with open("data/global_params.txt", "w") as f:
    f.write(global_params.to_json())

with open("data/key.txt", "w") as f:
    f.write(key.to_json())

with open("data/ciphertext.txt", "w") as f:
    f.write(ciphertext.to_json())

with open("data/message.txt", "w") as f:
    f.write(message)

with open ("data/base_circuit.qpy", "wb") as f:
    qpy_serialization.dump(original_circuit, f)

In [5]:
# Test 1 - delete
deletion_circuit_test1 = delete(ciphertext)
deletion_counts_test1 = run_and_measure(deletion_circuit_test1)
export_counts(deletion_counts_test1, csv_filename="data/test1-deletion-counts.csv", key_label="Deletion measurement")
verify_deletion_counts(deletion_counts_test1, key, global_params)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

Accepted proof of deletion: 902/1000 (90.2%)
Rejected proof of deletion: 98/1000 (9.8%)
Out of the 98 rejected certificates, the following are the counts of the Hamming distances between the received certificate and the expected certificate:
Hamming distance 1: 95
Hamming distance 2: 3


In [6]:
# Test 2 - decrypt
decryption_circuit_test2 = create_decryption_circuit(key, ciphertext)
decryption_counts_test2 = run_and_measure(decryption_circuit_test2)
export_counts(decryption_counts_test2, csv_filename="data/test2-decryption-counts.csv", key_label="Decryption measurement")
decrypt_results(decryption_counts_test2, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

Correct message decrypted: 867/1000 (86.7%)
Incorrect message decrypted: 133/1000 (13.3%)
Error detected during decryption process (hashes didn't match): 28/1000 (2.8000000000000003%)


In [7]:
# Test 3 - delete (possibly maliciously) and then decrypt
deletion_circuit_test3 = delete(ciphertext)
# deletion_circuit_test3 = breidbart_measurement(ciphertext)
ciphertext.circuit = deletion_circuit_test3 # To use the new circuit as the starting point of the decryption_measurement circuit
decryption_circuit_test3 = create_decryption_circuit(key, ciphertext)
raw_counts_test3 = run_and_measure(decryption_circuit_test3)
deletion_counts_test3 = {}
decryption_counts_test3 = {}
for measurement, count in raw_counts_test3.items():
    # Qiskit will return a space-separated string of the two measurements. 
    # run_and_measure will reverse the string so that the first substring is the first measurement.
    deletion_measurement, decryption_measurement = measurement.split(" ")
    deletion_counts_test3[deletion_measurement] = deletion_counts_test3.get(deletion_measurement, 0) + count
    decryption_counts_test3[decryption_measurement] = decryption_counts_test3.get(decryption_measurement, 0) + count

export_counts(raw_counts_test3, csv_filename="data/test3-raw-counts.csv", key_label="Measurement")
export_counts(deletion_counts_test3, csv_filename="data/test3-deletion-counts.csv", key_label="Deletion measurement")
export_counts(decryption_counts_test3, csv_filename="data/test3-decryption-counts.csv", key_label="Decryption measurement")
accepted_certificates_test3 = verify_deletion_counts(deletion_counts_test3, key, global_params)
# Test the decryption results for all the measurements, regardless of whether or not the certificate of deletion was accepted
decrypt_results(decryption_counts_test3, key, ciphertext, message)

# Test the decryption results for just the measurements where the certificate of deletion was accepted
accepted_deletion_decryption_counts_test3 = {}
for measurement, count in raw_counts_test3.items():
    deletion_measurement, decryption_measurement = measurement.split(" ")
    if deletion_measurement in accepted_certificates_test3:
        accepted_deletion_decryption_counts_test3[decryption_measurement] = accepted_deletion_decryption_counts_test3.get(decryption_measurement, 0) + count

print()
print("Of the measurements where the proof of deletion was accepted, the following are the decryption statistics:")
decrypt_results(accepted_deletion_decryption_counts_test3, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

Accepted proof of deletion: 897/1000 (89.7%)
Rejected proof of deletion: 103/1000 (10.299999999999999%)
Out of the 103 rejected certificates, the following are the counts of the Hamming distances between the received certificate and the expected certificate:
Hamming distance 1: 97
Hamming distance 2: 6
Correct message decrypted: 66/1000 (6.6000000000000005%)
Incorrect message decrypted: 934/1000 (93.4%)
Error detected during decryption process (hashes didn't match): 848/1000 (84.8%)

Of the measurements where the proof of deletion was accepted, the following are the decryption statistics:
Correct message decrypted: 63/897 (7.023411371237458%)
Incorrect message decrypted: 834/897 (92.97658862876254%)
Error detected during decryption process (hashes didn't match): 761/897 (84.83835005574136%)


In [8]:
# Create zip file
shutil.make_archive("data_export", "zip", "data");